In [1]:
import os, time, re, hashlib
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import warnings

warnings.simplefilter("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

print("Directorio de trabajo actual:", os.getcwd())

Directorio de trabajo actual: /workspaces/data-extraction


In [3]:
download_folder = Path("archivos")
download_folder.mkdir(parents=True, exist_ok=True)
for file in download_folder.glob("*"):
    file.unlink()


In [4]:
df_urls = pd.read_excel("/workspaces/data-extraction/Fuentes de información - Colombia.xlsx", sheet_name="DANE")
print(df_urls.head())


  Categoría                                      Variable  \
0  Economía                            Empleo y desempleo   
1  Economía  Indicador de Seguimiento a la Economía (ISE)   
2  Economía         Índice de Precios al Consumidor (IPC)   
3  Economía         Índice de Precios del Productor (IPP)   
4  Economía         Índice de Producción Industrial (IPI)   

                                                 URL  
0  https://www.dane.gov.co/index.php/estadisticas...  
1  https://www.dane.gov.co/index.php/estadisticas...  
2  https://www.dane.gov.co/index.php/estadisticas...  
3  https://www.dane.gov.co/index.php/estadisticas...  
4  https://www.dane.gov.co/index.php/estadisticas...  


In [5]:
all_data_dane = []
for idx, row in df_urls.iterrows():
    url = row['URL']
    category = row['Categoría']
    variable = row['Variable']
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Error accediendo a la URL: {url}")
            continue
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a')
        for link in links:
            href = link.get('href', '')
            if re.search(r"\.xlsx?$", href, re.IGNORECASE):
                from urllib.parse import urljoin
                if href.startswith("/"):
                    href = urljoin(url, href)
                title = link.get('title', '').strip()
                if title:
                    all_data_dane.append({
                        "Categoría": category,
                        "Variable": variable,
                        "Título": title,
                        "Enlace": href
                    })
    except Exception as e:
        print(f"Error procesando la URL {url}: {e}")

df_data = pd.DataFrame(all_data_dane)
df_data = df_data[df_data['Título'] != '']
df_data['Nombre_Archivo'] = df_data['Enlace'].apply(lambda x: os.path.basename(x).split('?')[0])
print(df_data.head())


  Categoría            Variable  \
0  Economía  Empleo y desempleo   
1  Economía  Empleo y desempleo   
2  Economía  Empleo y desempleo   
3  Economía  Empleo y desempleo   
4  Economía  Empleo y desempleo   

                                              Título  \
0         Anexo Mercado laboral febrero 2025 en xlsx   
1      Anexo desestacionalizado febrero 2025 en xlsx   
2                 Anexo actualización GEIH CNPV 2018   
3              Ver Indicadores de empleo 1958 - 2018   
4  Indicadores de empleo por categoría ocupaciona...   

                                              Enlace  \
0  https://www.dane.gov.co/files/operaciones/GEIH...   
1  https://www.dane.gov.co/files/operaciones/GEIH...   
2  https://www.dane.gov.co/files/investigaciones/...   
3  https://www.dane.gov.co/files/investigaciones/...   
4  https://www.dane.gov.co/files/investigaciones/...   

                                      Nombre_Archivo  
0                             anex-GEIH-feb2025.xlsx  
1    

In [6]:
print("Archivos a descargar:", len(df_data))
for idx, row in df_data.iterrows():
    file_url = row['Enlace']
    file_name = row['Nombre_Archivo']
    if not re.search(r"\.xlsx?$", file_url, re.IGNORECASE):
        continue
    download_path = download_folder / file_name
    try:
        with requests.get(file_url, stream=True) as r:
            r.raise_for_status()
            with open(download_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f"Archivo descargado exitosamente: {file_name}")
    except Exception as e:
        print(f"Error al descargar {file_name}: {e}")

print("Descarga de archivos completada.")


Archivos a descargar: 174
Archivo descargado exitosamente: anex-GEIH-feb2025.xlsx
Archivo descargado exitosamente: anex-GEIH-Desestacionalizado-feb2025.xlsx
Archivo descargado exitosamente: anexo-mercado-laboral-segun-proyecciones-CNPV2018.xlsx
Archivo descargado exitosamente: Ind_empleo_1958_2018.xls
Archivo descargado exitosamente: Ind_empleo_categoria_ocupacional_1991_2018.xls
Error al descargar Ind_empleo_grupo_ocupacion_1985_2018.xls: No connection adapters were found for 'sftp://mjbarraganc@172.200.0.23/files/investigaciones/boletines/ech/indicadores-competitividad/Ind_empleo_grupo_ocupacion_1985_2018.xls'
Archivo descargado exitosamente: Series_Est_Nal_empleo_1985-2018.xls
Archivo descargado exitosamente: anex-RELAB-empleo-dic2024.xlsx
Archivo descargado exitosamente: anex-RELAB-creativa-oct2024.xlsx
Archivo descargado exitosamente: Anexo_RELAB_naranja_ene_19_mar_22.xlsx
Archivo descargado exitosamente: Anexo_RELAB_naranja_ene_19_dic_21.xlsx
Archivo descargado exitosamente: Anex